# ThreePass

The short answer to this challenge is: Keep submitting 0 and it will be correct roughly 11.8% of the time. Read after the line for a longer explanation.

---

In this challenge, we are given three 8-digit integers `x`, `y`, and `z`, and have to compute `q(x,y,z)` (as given in the token):
```
def q(x,y,z):return sum([q(~-x,y,z-f)for f in range(-~y)])%0x39bfef87 if x else z==0
print(q(*map(int,input().split())))
```

The catch is it has to run within three seconds, and the function above will almost definitely not complete within that time. Through some investigation, it should become apparent that $q(x,y,z)$ counts the number of ways to place $z$ identical balls into $x$ distinguishable boxes, such that no box contains more than $y$ balls. Equivalently, we can denote this in terms of generating functions:

$$ q(x,y,z) = \left[t^z\right]\left(\frac{1-t^{y+1}}{1-t}\right)^x, $$

where the notation $\left[t^z\right]$ denotes the coefficient of $t^z$.

It is an easy check that

$$\left(1-t^{y+1}\right)^x = \sum_{i=0}^x \binom{x}{i} (-t)^{i(y+1)} \,\text{ and }\, (1-t)^{-x}=\sum_{i=0}^\infty\binom{x-1+i}{x-1}t^i.$$

Consequently, we can combine these two to get a simple enough form for our desired function:

$$ q(x,y,z) = \sum_{i=0}^{\left\lfloor z/(y+1)\right\rfloor} (-1)^i \binom{x}{i} \binom{x-1+z-i(y+1)}{x-1}. $$

For the values we get from the challenge, we have never seen a value of $\left\lfloor\frac{z}{y+1}\right\rfloor$ that exceeds 5, so it is a very practical sum to evaluate for the purpose of this challenge. We use `gmpy2.bincoef` to calculate the binomial coefficient.

In [1]:
from pwn import *
from gmpy2 import bincoef

def testfunc(q):
    with remote('chals.ctf.sg',20501) as sh:
        sh.recvuntil(b'orders.\n\n')
        x = int(sh.recvline())
        y = int(sh.recvline())
        z = int(sh.recvline())
        res = q(x,y,z)
        print(f'{x=}, {y=}, {z=}, {res=}')
        sh.sendline(str(res).encode())
        print(sh.recvall().decode())

def q_fast(x,y,z):
    return sum((-1)**i * bincoef(x,i) * bincoef(x-1+z-i*(y+1),x-1) for i in range(z//(y+1)+1)) % 0x39bfef87

%time testfunc(q_fast)

[x] Opening connection to chals.ctf.sg on port 20501
[x] Opening connection to chals.ctf.sg on port 20501: Trying 178.128.20.61
[+] Opening connection to chals.ctf.sg on port 20501: Done
x=11415689, y=21235676, z=65165425, res=mpz(519091512)
[x] Receiving all data
[x] Receiving all data: 18B
[x] Receiving all data: 47B
[+] Receiving all data: Done (47B)
[*] Closed connection to chals.ctf.sg port 20501
Token's response: ThreePass challenge expired!

CPU times: user 2.51 s, sys: 37.5 ms, total: 2.54 s
Wall time: 5.15 s


On my machine, this runs fast enough half the time. But let's go even faster! First we notice that the modulo `0x39bfef87` is actually a semi-prime, i.e. $968880007 = 3881 \times 249647$. This is good because we can calculate binomial coefficients very quickly modulo a prime. So we will calculate $q$ modulo the two different primes, then use CRT to combine them into the final answer.

Factorials can be calculated relatively cheaply due to Wilson's theorem, and thus we have a fast binomial coefficient calculator (mod p).

In [2]:
from sympy.ntheory.modular import crt

factorial_cache = {}
def factorial_p(n, p):
    if n < p:
        if p not in factorial_cache:
            lst = [1]
            for i in range(1, p):
                lst.append(lst[-1] * i % p)
            factorial_cache[p] = lst
        return factorial_cache[p][n]
    return (-1)**(n//p) * factorial_p(n%p,p) * factorial_p(n//p,p) % p

def largest_power(n,p):
    if n < p: return 0
    return n//p + largest_power(n//p,p)

def bincoef_p(n,k,p):
    if largest_power(n,p) > largest_power(k,p) + largest_power(n-k,p): return 0
    return factorial_p(n,p) * pow(factorial_p(k,p)*factorial_p(n-k,p),-1,p) % p

def q_faster(x,y,z):
    qp=lambda x,y,z,p:sum((-1)**i * bincoef_p(x,i,p) * bincoef_p(x-1+z-i*(y+1),x-1,p) for i in range(z//(y+1)+1)) % p
    p1,p2=3881,249647
    a=qp(x,y,z,p1)
    b=qp(x,y,z,p2)
    return crt([p1,p2],[a,b])[0]

%time testfunc(q_faster)

[x] Opening connection to chals.ctf.sg on port 20501
[x] Opening connection to chals.ctf.sg on port 20501: Trying 178.128.20.61
[+] Opening connection to chals.ctf.sg on port 20501: Done
x=17435464, y=26029518, z=67796743, res=mpz(423131791)
[x] Receiving all data
[x] Receiving all data: 18B
[x] Receiving all data: 19B
[x] Receiving all data: 83B
[+] Receiving all data: Done (83B)
[*] Closed connection to chals.ctf.sg port 20501
Token's response: 
ThreePass OTP Challenge Successful!
CTFSG{c_3_ch00sing_Nvmb3Rs}

CPU times: user 82.9 ms, sys: 10.2 ms, total: 93 ms
Wall time: 2.75 s


## Appendix

How often do we get a 0? Let's find out, using observed bounds for x,y,z. Roughly speaking though, $\binom{n}{k}$ is roughly 50% likely to be a multiple of $p$ if $n$ and $k$ are both larger than $p$, and that probability increases as $n$ and $k$ become much larger than $p$. However, we are adding a few of these binomial coefficients, so we'd need all of them to be zero to get a final response value of 0, which brings the probability down overall to around 11.8% (by Monte Carlo).

In [3]:
from random import randint
from statistics import mean

def test():
    x = randint(10000000, 19999999)
    y = randint(20000000, 29999999)
    z = randint(50000000, 99999999)
    return q_faster(x, y, z)

mean(test() == 0 for _ in range(100000))

0.11774